# APIs in Python

## Import Statements

In [5]:
import requests
import json
from datetime import datetime
import pandas as pd
import time

## Get a Google Maps API Key

Some parts of this project require a **Google Maps API key**.

### Step 1: Create a Google Cloud Project
1. Go to:  
   https://console.cloud.google.com/
2. Sign in with your Google account.
3. Create a **new project** (or select an existing one).

### Step 2: Enable the Places API
1. In the Google Cloud Console, go to **APIs & Services → Library**.
2. Search for **Places API** (or Nearby Search API).
3. Click **Enable**.

### Step 3: Create an API Key
1. Go to **APIs & Services → Credentials**.
2. Click **Create Credentials → API Key**.
3. Copy your API key and keep it private.

> **Important:** Google may require billing information, but the free tier is sufficient for this lesson.

### Retrieving Existing API Key
1. Go to **APIs & Services → Credentials**.
2. Click **Show Key**

## Copy Your API Key into this string
If you plan to publish code using APIs do **not** include your API key

In [3]:
API_KEY = ''

## Searching Restaurants Given a Location
The following code block gets data for restaurants given a location and radius.
Notes:
- The first 2 lines define the location in longitude and latitude coordinates and the radius in meters from that point where our search will take place
- Each request is capped at 20 restaurants, for the sake of this excercise this is sufficient
- The output is a json file, we will learn more about reading json files later in this notebook
- The name of the output file includes the date and time. This was added to prevent accidentally overriding data as every time the code runs it would produce a file with a unique name. This is up to the user's preference and can be changed.

### Note
Do not run the code over and over again excessively, there are limits to the amount of times you can use the API before being charged (about 6,000 requests per month)

In [44]:
LOCATION = '40.7644,-73.9184' #Feel free to change this
RADIUS = 2000  # in meters, feel free to change this too

# Using the API
url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={LOCATION}&radius={RADIUS}&type=restaurant&key={API_KEY}"
response = requests.get(url)
restaurants = response.json().get('results', [])

# This part creates the file name with the time stamp. Feel free to change this if you would like.
# Important - make sure your filename ends in .json
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
filename = f'restaurants_{timestamp}.json'
   
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(restaurants, f, indent=2)
    
print(f"Saved {len(restaurants)} restaurants to {filename}")

Saved 20 restaurants to restaurants_2025-12-29_23-08-26.json


#### Check
Now let's check to see if `restaurants` has stored our data.
We should also check the folder this file is in to see if the file was created.

In [20]:
# Let's look at the type of our, it should be a list
type(restaurants)

list

In [49]:
# Assuming it is a list, let's look at the first element
print(f"This should be a dictionary: {type(restaurants[0])}")
restaurants[0]

This should be a dictionary: <class 'dict'>


{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 40.7555621, 'lng': -73.9224658},
  'viewport': {'northeast': {'lat': 40.7569783802915,
    'lng': -73.9213070197085},
   'southwest': {'lat': 40.7542804197085, 'lng': -73.9240049802915}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
 'icon_background_color': '#FF9E67',
 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet',
 'international_phone_number': '+1 718-943-7404',
 'name': "Applebee's Grill + Bar",
 'opening_hours': {'open_now': True},
 'photos': [{'height': 3024,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/107013409457296046665">Carlos Vega</a>'],
   'photo_reference': 'AZLasHrwXCaeVywkRpBf40i6-vVwsg27htIXMtUBgitzkITYW5ykVjyoFWII18TARtIMwu-CTcli-jjmmfKjaQr4UdmCASPOqADVQhYOgEaNx7eEEGANl-ojKXdCqFi72TCgyx-QWGer1Bclid3u1OCwI0iQE3F2Ift8uwtWh8HuR1Wb7Ir-QI7Rmh0IrMqcZQ1Ea9L-o1FvjKeW3Sjna_uQPJfxc2Hao35Gx

#### Result
The above code should produce something like this

{'business_status': 'OPERATIONAL',<br>
 'geometry': {'location': {'lat': 40.7555621, 'lng': -73.9224658},<br>
  'viewport': {'northeast': {'lat': 40.7569783802915,<br>
    'lng': -73.9213070197085},<br>
...
}

You can also check the other elements of `restuarants` follow a similar format.

We will go further into analyzing the data later in the lesson

#### Reading Files
The output said that our data was saved to a .json file. Let's practice reading json files

This cell should output a list of dictionaries similar to the one above.

In [53]:
file_name = "example_file.json" # You can replace this with your file name
with open(file_name) as f:
    data_from_json = json.load(f)
data_from_json[0] #You can change this to look at different elements

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 40.7555621, 'lng': -73.9224658},
  'viewport': {'northeast': {'lat': 40.7569783802915,
    'lng': -73.9213070197085},
   'southwest': {'lat': 40.7542804197085, 'lng': -73.9240049802915}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
 'icon_background_color': '#FF9E67',
 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet',
 'international_phone_number': '+1 718-943-7404',
 'name': "Applebee's Grill + Bar",
 'opening_hours': {'open_now': True},
 'photos': [{'height': 3024,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/107013409457296046665">Carlos Vega</a>'],
   'photo_reference': 'AZLasHrwXCaeVywkRpBf40i6-vVwsg27htIXMtUBgitzkITYW5ykVjyoFWII18TARtIMwu-CTcli-jjmmfKjaQr4UdmCASPOqADVQhYOgEaNx7eEEGANl-ojKXdCqFi72TCgyx-QWGer1Bclid3u1OCwI0iQE3F2Ift8uwtWh8HuR1Wb7Ir-QI7Rmh0IrMqcZQ1Ea9L-o1FvjKeW3Sjna_uQPJfxc2Hao35Gx

## Getting Coordinates from a Zip Code
The above code is dependent on having the longitude and latitude coordinates for the center of your search. Often times, we may not have that but instead have a zip code. We can get longitude and latitude coordinates from a zip code using Google's Geocoding API.

We first need to enable it using the following steps:
- Go to https://console.cloud.google.com/
- Make sure the correct project is selected (top bar).
- Navigate to APIs & Services → Library
- Search for Geocoding API
- Click Enable

In [57]:
ZIP_CODE = "10001"

url = f"https://maps.googleapis.com/maps/api/geocode/json?address={ZIP_CODE}&key={API_KEY}"
response = requests.get(url)
data = response.json()
if data['results']:
    location = data['results'][0]['geometry']['location']
    lat = location['lat']
    lng = location['lng']
    print(f"Coordinates for {ZIP_CODE}: {round(lat,3)}, {round(lng,3)}")
else:
    print("No results found")

Coordinates for 10001: 40.754, -73.999


## Getting Data from a Specific Address
Suppose we wanted to get rating data for a policy we plan to write or have already written. We would probably already have the address and name of the business. Let's get data from the API given a list of specific locations.

#### Note
When looking up restaurants and other businesses, the API works better if you put the name of the business in the search rather than the building number. This is because the API is designed to search for prominent locations which are primarily identified by their unique names rather than just raw address data.

In [42]:
# Addresses that we will be testing. Feel free to add and/or remove items
addresses = [
    "Gino’s Pizzeria and Restaurant Astoria, NY 11103",
    "43-19 Broadway, Astoria, NY 11103", #This is the same place as the item above. I included it to demonstrate the note
    "Applebee's Grill + Bar 35th Avenue, Astoria, NY 11101",
    "DiWine Natural Wine Bar & Restaurant, Astoria, NY 11103",
    "Spyce Astoria, Astoria, NY 11103",
    "German Doner Kebab Steinway St, Astoria, NY 11103",
    "Chuck E. Cheese 48th Street, Long Island City"
]
    
def get_place_data(address):
    '''
    This function returns a place id from an address. The address must be a string.
    '''
    url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
    params = {
        "input": address,
        "inputtype": "textquery",
        "fields": "place_id",
        "key": API_KEY
    }
    response = requests.get(url, params=params)
    return response.json()
    
def get_place_details(place_id):
    '''
    This function returns a dictionary of place data given a place_id.

    Within params, "fields" can be changed to include or exclude certain data.
      > All fields need to be put into the string separated by commas
      > All field names must be from the predefined list defined by the API.
      
    You can find the names of valid fields here https://developers.google.com/maps/documentation/places/web-service/place-details#fieldmask
    '''
    url = "https://maps.googleapis.com/maps/api/place/details/json"
    params = {
        "place_id": place_id,
        "fields": "name,formatted_address,types,website,international_phone_number,business_status", # Feel free to edit this
        "key": API_KEY
    }
    response = requests.get(url, params=params)
    return response.json().get("result", {})


results = []
for addr in addresses:
    print(f"Searching: {addr}")

    # Using our first function to get the place_id
    data = get_place_data(addr)
    candidates = data.get("candidates", [])

    # The code below checks to see if the candidates list is empty. If it is not empty, it takes the first place_id and gets the details from it
    # There may be more sophisticated ways of finding which candidate is the best to fine tune the search 
    # For this lesson, this method is sufficient
    if candidates:
        place_id = candidates[0]['place_id']
        details = get_place_details(place_id)
        results.append(details)
    else:
        print("No match found.")
    time.sleep(0.2)  # Respect rate limits

timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
filename = f'restaurants_by_address_{timestamp}.json'
  
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2)
   
print(f"Saved {len(results)} restaurants to {filename}")

Searching: Gino’s Pizzeria and Restaurant Astoria, NY 11103
Searching: 43-19 Broadway, Astoria, NY 11103
Searching: Applebee's Grill + Bar 35th Avenue, Astoria, NY 11101
Searching: DiWine Natural Wine Bar & Restaurant, Astoria, NY 11103
Searching: Spyce Astoria, Astoria, NY 11103
Searching: German Doner Kebab Steinway St, Astoria, NY 11103
Searching: Chuck E. Cheese 48th Street, Long Island City
Saved 7 restaurants to restaurants_by_address_2025-12-29_23-05-04.json


# Cleaning and Interpreting Data

We have now been over multiple ways of pulling data from the API.

We will now go over methods to make our data easier to understand and use.

In [136]:
# Pick one of the json files you made in the code above
# You can also use the pre-made example file
file_name = 'example_file.json'
with open(file_name) as f:
    data_from_json = json.load(f)
data_from_json[0]

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 40.7555621, 'lng': -73.9224658},
  'viewport': {'northeast': {'lat': 40.7569783802915,
    'lng': -73.9213070197085},
   'southwest': {'lat': 40.7542804197085, 'lng': -73.9240049802915}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
 'icon_background_color': '#FF9E67',
 'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v2/restaurant_pinlet',
 'international_phone_number': '+1 718-943-7404',
 'name': "Applebee's Grill + Bar",
 'opening_hours': {'open_now': True},
 'photos': [{'height': 3024,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/107013409457296046665">Carlos Vega</a>'],
   'photo_reference': 'AZLasHrwXCaeVywkRpBf40i6-vVwsg27htIXMtUBgitzkITYW5ykVjyoFWII18TARtIMwu-CTcli-jjmmfKjaQr4UdmCASPOqADVQhYOgEaNx7eEEGANl-ojKXdCqFi72TCgyx-QWGer1Bclid3u1OCwI0iQE3F2Ift8uwtWh8HuR1Wb7Ir-QI7Rmh0IrMqcZQ1Ea9L-o1FvjKeW3Sjna_uQPJfxc2Hao35Gx

In [137]:
# Let's check to make sure our data types are all right
print(f"This should be a list: {type(data_from_json)}")
print(f"This should be a dict: {type(data_from_json[0])}")

This should be a list: <class 'list'>
This should be a dict: <class 'dict'>


## Cleaning up the Data using Pandas

In [159]:
# We can convert our data from the list of dictionaries to a dataframe
pd.DataFrame(data=data_from_json)

,business_status,geometry,icon,icon_background_color,icon_mask_base_uri,international_phone_number,name,opening_hours,photos,place_id,plus_code,price_level,rating,reference,scope,types,user_ratings_total,vicinity,permanently_closed
0,OPERATIONAL,"{'location': {'lat': 40.7555621, 'lng': -73.92...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-943-7404,Applebee's Grill + Bar,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJ8XWaOSVfwokR3LzQfMwbmGU,"{'compound_code': 'Q34H+62 Astoria, Queens, NY...",2.0,4.0,ChIJ8XWaOSVfwokR3LzQfMwbmGU,GOOGLE,"[restaurant, bar, food, point_of_interest, est...",2322,"38-01 35th Avenue, Astoria",NaN
1,OPERATIONAL,"{'location': {'lat': 40.7602103, 'lng': -73.91...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-313-0404,Spyce Astoria,{'open_now': False},"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJ_eRvfz5fwokRE0sUHVm_UGY,"{'compound_code': 'Q36M+3V Astoria, Queens, NY...",3.0,3.1,ChIJ_eRvfz5fwokRE0sUHVm_UGY,GOOGLE,"[night_club, bar, restaurant, food, point_of_i...",192,"42-18 31st Avenue, Astoria",NaN
2,OPERATIONAL,"{'location': {'lat': 40.7744143, 'lng': -73.90...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-777-8830,Tasty's Diner,{'open_now': False},"[{'height': 3000, 'html_attributions': ['<a hr...",ChIJzwijNGZfwokRl6h8wS-1MuY,"{'compound_code': 'Q3FR+QG Astoria, Queens, NY...",1.0,4.1,ChIJzwijNGZfwokRl6h8wS-1MuY,GOOGLE,"[meal_delivery, restaurant, food, point_of_int...",580,"35-12 Ditmars Boulevard, Astoria",NaN
3,OPERATIONAL,"{'location': {'lat': 40.76240990000001, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-545-4775,Dino's Pizzeria,{'open_now': False},"[{'height': 2269, 'html_attributions': ['<a hr...",ChIJeWtbNzdfwokRQdWeC8tUJtI,"{'compound_code': 'Q36F+XJ Astoria, Queens, NY...",1.0,4.2,ChIJeWtbNzdfwokRQdWeC8tUJtI,GOOGLE,"[meal_delivery, restaurant, food, point_of_int...",381,"30-01 Broadway, Astoria",NaN
4,OPERATIONAL,"{'location': {'lat': 40.7608158, 'lng': -73.91...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-777-1355,DiWine Natural Wine Bar & Restaurant,{'open_now': True},"[{'height': 1365, 'html_attributions': ['<a hr...",ChIJB8q5hz5fwokRycD7DHYjj7Y,"{'compound_code': 'Q36M+8P Astoria, Queens, NY...",2.0,4.6,ChIJB8q5hz5fwokRycD7DHYjj7Y,GOOGLE,"[bar, restaurant, food, point_of_interest, est...",564,"41-15 31st Avenue, Astoria",NaN
5,OPERATIONAL,"{'location': {'lat': 40.7644274, 'lng': -73.91...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-278-1931,Sampan,{'open_now': False},"[{'height': 4000, 'html_attributions': ['<a hr...",ChIJPftdmj9fwokRAeY5lVdwWi8,"{'compound_code': 'Q37P+Q3 Astoria, Queens, NY...",1.0,4.0,ChIJPftdmj9fwokRAeY5lVdwWi8,GOOGLE,"[meal_delivery, restaurant, food, point_of_int...",143,"28-48 Steinway Street, Astoria",NaN
6,OPERATIONAL,"{'location': {'lat': 40.7743819, 'lng': -73.90...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-545-8666,Taverna Kyclades Astoria,{'open_now': False},"[{'height': 3000, 'html_attributions': ['<a hr...",ChIJ90pvtWdfwokRw86tuNLCG7g,"{'compound_code': 'Q3FR+QR Astoria, Queens, NY...",2.0,4.6,ChIJ90pvtWdfwokRw86tuNLCG7g,GOOGLE,"[restaurant, food, point_of_interest, establis...",5065,"36-01 Ditmars Boulevard, Astoria",NaN
7,OPERATIONAL,"{'location': {'lat': 40.758773, 'lng': -73.918...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 347-639-1435,Pollos A La Brasa Mario,{'open_now': False},"[{'height': 2048, 'html_attributions': ['<a hr...",ChIJF8BDeDxfwok

In [160]:
# We will save the dataframe as df
df = pd.DataFrame(data=data_from_json)

In [161]:
df.columns

Index(['business_status', 'geometry', 'icon', 'icon_background_color',
       'icon_mask_base_uri', 'international_phone_number', 'name',
       'opening_hours', 'photos', 'place_id', 'plus_code', 'price_level',
       'rating', 'reference', 'scope', 'types', 'user_ratings_total',
       'vicinity', 'permanently_closed'],
      dtype='object')

In [162]:
# Let's move the 'name' column to be the first in the list
columns_without_name = [c for c in df.columns if c!='name'] #all columns besides name in the same order
reordered_columns = ['name']+columns_without_name # name and then all other columns
df = df[reordered_columns]
df.head()

,name,business_status,geometry,icon,icon_background_color,icon_mask_base_uri,international_phone_number,opening_hours,photos,place_id,plus_code,price_level,rating,reference,scope,types,user_ratings_total,vicinity,permanently_closed
0,Applebee's Grill + Bar,OPERATIONAL,"{'location': {'lat': 40.7555621, 'lng': -73.92...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-943-7404,{'open_now': True},"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJ8XWaOSVfwokR3LzQfMwbmGU,"{'compound_code': 'Q34H+62 Astoria, Queens, NY...",2.0,4.0,ChIJ8XWaOSVfwokR3LzQfMwbmGU,GOOGLE,"[restaurant, bar, food, point_of_interest, est...",2322,"38-01 35th Avenue, Astoria",NaN
1,Spyce Astoria,OPERATIONAL,"{'location': {'lat': 40.7602103, 'lng': -73.91...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-313-0404,{'open_now': False},"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJ_eRvfz5fwokRE0sUHVm_UGY,"{'compound_code': 'Q36M+3V Astoria, Queens, NY...",3.0,3.1,ChIJ_eRvfz5fwokRE0sUHVm_UGY,GOOGLE,"[night_club, bar, restaurant, food, point_of_i...",192,"42-18 31st Avenue, Astoria",NaN
2,Tasty's Diner,OPERATIONAL,"{'location': {'lat': 40.7744143, 'lng': -73.90...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-777-8830,{'open_now': False},"[{'height': 3000, 'html_attributions': ['<a hr...",ChIJzwijNGZfwokRl6h8wS-1MuY,"{'compound_code': 'Q3FR+QG Astoria, Queens, NY...",1.0,4.1,ChIJzwijNGZfwokRl6h8wS-1MuY,GOOGLE,"[meal_delivery, restaurant, food, point_of_int...",580,"35-12 Ditmars Boulevard, Astoria",NaN
3,Dino's Pizzeria,OPERATIONAL,"{'location': {'lat': 40.76240990000001, 'lng':...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-545-4775,{'open_now': False},"[{'height': 2269, 'html_attributions': ['<a hr...",ChIJeWtbNzdfwokRQdWeC8tUJtI,"{'compound_code': 'Q36F+XJ Astoria, Queens, NY...",1.0,4.2,ChIJeWtbNzdfwokRQdWeC8tUJtI,GOOGLE,"[meal_delivery, restaurant, food, point_of_int...",381,"30-01 Broadway, Astoria",NaN
4,DiWine Natural Wine Bar & Restaurant,OPERATIONAL,"{'location': {'lat': 40.7608158, 'lng': -73.91...",https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-777-1355,{'open_now': True},"[{'height': 1365, 'html_attributions': ['<a hr...",ChIJB8q5hz5fwokRycD7DHYjj7Y,"{'compound_code': 'Q36M+8P Astoria, Queens, NY...",2.0,4.6,ChIJB8q5hz5fwokRycD7DHYjj7Y,GOOGLE,"[bar, restaurant, food, point_of_interest, est...",564,"41-15 31st Avenue, Astoria",NaN


### Flattening Dictionaries
Some of our columns have data stored as dictionaries. Some of these dictionaries have dictionaries inside of them. Let's separate the data by column and key to make the data more readable.

In [163]:
def flatten_dict(d, parent_key=""):
    """
    Convert a nested dictionary into a single-level dictionary.

    This function takes a dictionary that may contain other dictionaries
    as values and "flattens" it so that there are no nested dictionaries.
    The keys from inner dictionaries are combined with their parent keys
    using underscores.

    Example:
        Input:
            {"geometry": {"location": {"lat": 40.7,"lng": -73.9}}}

        Output:
            {
                "geometry_location_lat": 40.7,
                "geometry_location_lng": -73.9
            }

    Parameters:
        d (dict):
            The dictionary to flatten. It may contain nested dictionaries.
        parent_key (str):
            Used internally during recursion to build up the full key name.
            When calling the function yourself, you should not set this.
    """
    items = {}

    # Loop through each key-value pair in the dictionary
    for k, v in d.items():
        # Create a new key name.
        # If we are inside a nested dictionary, prepend the parent key.
        # Otherwise, just use the current key.
        new_key = f"{parent_key}_{k}" if parent_key else k

        # If the value is another dictionary, flatten it recursively
        if isinstance(v, dict):
            # Flatten the inner dictionary and add it to the items
            items.update(flatten_dict(v, new_key))
        else:
            # If the value is not a dictionary, store it directly
            items[new_key] = v
    return items


df_expanded = df.copy()

for col in df.columns:
    if df[col].apply(lambda x: isinstance(x, dict)).any():
        flattened = df[col].apply(lambda x: flatten_dict(x) if isinstance(x, dict) else {})
        new_cols = pd.json_normalize(flattened).add_prefix(f"{col}_")
        df_expanded = pd.concat([df_expanded, new_cols], axis=1)


df = df_expanded.drop(
    columns=[col for col in df.columns if df[col].apply(lambda x: isinstance(x, dict)).any()] # Remove the columns with dictionaries in them
)
df.head()

,name,business_status,icon,icon_background_color,icon_mask_base_uri,international_phone_number,photos,place_id,price_level,rating,...,permanently_closed,geometry_location_lat,geometry_location_lng,geometry_viewport_northeast_lat,geometry_viewport_northeast_lng,geometry_viewport_southwest_lat,geometry_viewport_southwest_lng,opening_hours_open_now,plus_code_compound_code,plus_code_global_code
0,Applebee's Grill + Bar,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-943-7404,"[{'height': 3024, 'html_attributions': ['<a hr...",ChIJ8XWaOSVfwokR3LzQfMwbmGU,2.0,4.0,...,NaN,40.755562,-73.922466,40.756978,-73.921307,40.754280,-73.924005,True,"Q34H+62 Astoria, Queens, NY, USA",87G8Q34H+62
1,Spyce Astoria,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-313-0404,"[{'height': 4032, 'html_attributions': ['<a hr...",ChIJ_eRvfz5fwokRE0sUHVm_UGY,3.0,3.1,...,NaN,40.760210,-73.915277,40.761664,-73.913876,40.758966,-73.916574,False,"Q36M+3V Astoria, Queens, NY, USA",87G8Q36M+3V
2,Tasty's Diner,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-777-8830,"[{'height': 3000, 'html_attributions': ['<a hr...",ChIJzwijNGZfwokRl6h8wS-1MuY,1.0,4.1,...,NaN,40.774414,-73.908738,40.775862,-73.907291,40.773164,-73.909989,False,"Q3FR+QG Astoria, Queens, NY, USA",87G8Q3FR+QG
3,Dino's Pizzeria,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-545-4775,"[{'height': 2269, 'html_attributions': ['<a hr...",ChIJeWtbNzdfwokRQdWeC8tUJtI,1.0,4.2,...,NaN,40.762410,-73.925986,40.763706,-73.924684,40.761008,-73.927382,False,"Q36F+XJ Astoria, Queens, NY, USA",87G8Q36F+XJ
4,DiWine Natural Wine Bar & Restaurant,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,#FF9E67,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-777-1355,"[{'height': 1365, 'html_attributions': ['<a hr...",ChIJB8q5hz5fwokRycD7DHYjj7Y,2.0,4.6,...,NaN,40.760816,-73.915638,40.762098,-73.914343,40.759400,-73.917041,True,"Q36M+8P Astoria, Queens, NY, USA",87G8Q36M+8P


### Unique Values
Columns that only have 1 unique value are most likely not useful. Let's write code to see the columns with only 1 unique value.

In [164]:
# Let's see the number of unique values per column
for col in df.columns:
    print()
    uniques = set(df[col].astype(str))
    print(f"{col}: {len(uniques)}")
    if(len(uniques)==1):
        print('UNIQUE VALUE: '+str(uniques))


name: 20

business_status: 2

icon: 3

icon_background_color: 2

icon_mask_base_uri: 3

international_phone_number: 20

photos: 20

place_id: 20

price_level: 4

rating: 11

reference: 20

scope: 1
UNIQUE VALUE: {'GOOGLE'}

types: 8

user_ratings_total: 20

vicinity: 20

permanently_closed: 2

geometry_location_lat: 20

geometry_location_lng: 20

geometry_viewport_northeast_lat: 20

geometry_viewport_northeast_lng: 20

geometry_viewport_southwest_lat: 20

geometry_viewport_southwest_lng: 20

opening_hours_open_now: 3

plus_code_compound_code: 20

plus_code_global_code: 20


### Drop Unnecessary Columns

In [166]:
# Dropping unnecessary columns
df = df.drop(columns=['scope','icon','icon_background_color','photos']) # Feel free to add or change this
df.head()

,name,business_status,icon_mask_base_uri,international_phone_number,place_id,price_level,rating,reference,types,user_ratings_total,...,permanently_closed,geometry_location_lat,geometry_location_lng,geometry_viewport_northeast_lat,geometry_viewport_northeast_lng,geometry_viewport_southwest_lat,geometry_viewport_southwest_lng,opening_hours_open_now,plus_code_compound_code,plus_code_global_code
0,Applebee's Grill + Bar,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-943-7404,ChIJ8XWaOSVfwokR3LzQfMwbmGU,2.0,4.0,ChIJ8XWaOSVfwokR3LzQfMwbmGU,"[restaurant, bar, food, point_of_interest, est...",2322,...,NaN,40.755562,-73.922466,40.756978,-73.921307,40.754280,-73.924005,True,"Q34H+62 Astoria, Queens, NY, USA",87G8Q34H+62
1,Spyce Astoria,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-313-0404,ChIJ_eRvfz5fwokRE0sUHVm_UGY,3.0,3.1,ChIJ_eRvfz5fwokRE0sUHVm_UGY,"[night_club, bar, restaurant, food, point_of_i...",192,...,NaN,40.760210,-73.915277,40.761664,-73.913876,40.758966,-73.916574,False,"Q36M+3V Astoria, Queens, NY, USA",87G8Q36M+3V
2,Tasty's Diner,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-777-8830,ChIJzwijNGZfwokRl6h8wS-1MuY,1.0,4.1,ChIJzwijNGZfwokRl6h8wS-1MuY,"[meal_delivery, restaurant, food, point_of_int...",580,...,NaN,40.774414,-73.908738,40.775862,-73.907291,40.773164,-73.909989,False,"Q3FR+QG Astoria, Queens, NY, USA",87G8Q3FR+QG
3,Dino's Pizzeria,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-545-4775,ChIJeWtbNzdfwokRQdWeC8tUJtI,1.0,4.2,ChIJeWtbNzdfwokRQdWeC8tUJtI,"[meal_delivery, restaurant, food, point_of_int...",381,...,NaN,40.762410,-73.925986,40.763706,-73.924684,40.761008,-73.927382,False,"Q36F+XJ Astoria, Queens, NY, USA",87G8Q36F+XJ
4,DiWine Natural Wine Bar & Restaurant,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-777-1355,ChIJB8q5hz5fwokRycD7DHYjj7Y,2.0,4.6,ChIJB8q5hz5fwokRycD7DHYjj7Y,"[bar, restaurant, food, point_of_interest, est...",564,...,NaN,40.760816,-73.915638,40.762098,-73.914343,40.759400,-73.917041,True,"Q36M+8P Astoria, Queens, NY, USA",87G8Q36M+8P


### Making Binary Variables for `types`
`types` is a column in our dataset containing a list of descriptors for the establishment.

For each descriptor, we can make a binary variable equal to `1` if the establishment contains the descriptor and `0` otherwise.

In [167]:
# First, let's find all unique values within the types
unique_types = set()
for sublist in df['types']:
    unique_types = unique_types | set(sublist) # This is the union of 2 sets
unique_types

{'bakery',
 'bar',
 'cafe',
 'establishment',
 'food',
 'meal_delivery',
 'meal_takeaway',
 'night_club',
 'point_of_interest',
 'restaurant',
 'store'}

In [168]:
for restaurant_type in unique_types:
    binary_values = []
    for type_list in df['types']:
        if(restaurant_type in type_list):
            binary_values.append(1)
        else:
            binary_values.append(0)
    df[restaurant_type] = binary_values
df.head()

,name,business_status,icon_mask_base_uri,international_phone_number,place_id,price_level,rating,reference,types,user_ratings_total,...,bar,night_club,food,meal_takeaway,restaurant,establishment,meal_delivery,cafe,store,point_of_interest
0,Applebee's Grill + Bar,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-943-7404,ChIJ8XWaOSVfwokR3LzQfMwbmGU,2.0,4.0,ChIJ8XWaOSVfwokR3LzQfMwbmGU,"[restaurant, bar, food, point_of_interest, est...",2322,...,1,0,1,0,1,1,0,0,0,1
1,Spyce Astoria,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-313-0404,ChIJ_eRvfz5fwokRE0sUHVm_UGY,3.0,3.1,ChIJ_eRvfz5fwokRE0sUHVm_UGY,"[night_club, bar, restaurant, food, point_of_i...",192,...,1,1,1,0,1,1,0,0,0,1
2,Tasty's Diner,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-777-8830,ChIJzwijNGZfwokRl6h8wS-1MuY,1.0,4.1,ChIJzwijNGZfwokRl6h8wS-1MuY,"[meal_delivery, restaurant, food, point_of_int...",580,...,0,0,1,0,1,1,1,0,0,1
3,Dino's Pizzeria,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-545-4775,ChIJeWtbNzdfwokRQdWeC8tUJtI,1.0,4.2,ChIJeWtbNzdfwokRQdWeC8tUJtI,"[meal_delivery, restaurant, food, point_of_int...",381,...,0,0,1,0,1,1,1,0,0,1
4,DiWine Natural Wine Bar & Restaurant,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,+1 718-777-1355,ChIJB8q5hz5fwokRycD7DHYjj7Y,2.0,4.6,ChIJB8q5hz5fwokRycD7DHYjj7Y,"[bar, restaurant, food, point_of_interest, est...",564,...,1,0,1,0,1,1,0,0,0,1
